# Azure AI Search with LlamaIndex and TruLens

This notebook demonstrates the use of Azure AI Search in conjunction with OpenAI, Ragas, and Langchain. The goal is to evaluate a Langchain AI Search vector store based on the following metrics:

- **Context Precision**: Measures whether the relevant chunks for answering the question are present in the context and ranked highly.
- **Context Recall**: Assesses whether the retrieved chunks align with the ground truth answer in our evaluation test set of questions.

## Introduction to RAG Triad by TruLens

RAGs (Retrieval-Augmented Generation) have become the standard architecture for providing LLMs (Large Language Models) with context to avoid hallucinations. However, even RAGs can suffer from hallucination when retrieval fails to provide sufficient or relevant context. TruEra's RAG Triad evaluates hallucinations along three edges of the RAG architecture:

### Context Relevance
The first step of any RAG application is retrieval; to verify the quality of our retrieval, we want to make sure that each chunk of context is relevant to the input query. This is critical because this context will be used by the LLM to form an answer, so any irrelevant information in the context could be weaved into a hallucination. TruLens enables you to evaluate context relevance by using the structure of the serialized record.

### Groundedness
After the context is retrieved, it is then formed into an answer by an LLM. LLMs are often prone to stray from the facts provided, exaggerating or expanding to a correct-sounding answer. To verify the groundedness of our application, we can separate the response into individual claims and independently search for evidence that supports each within the retrieved context.

### Answer Relevance
Our response still needs to helpfully answer the original question. We can verify this by evaluating the relevance of the final response to the user input.

### Putting it together
By reaching satisfactory evaluations for this triad, we can make a nuanced statement about our application’s correctness; our application is verified to be hallucination-free up to the limit of its knowledge base. In other words, if the vector database contains only accurate information, then the answers provided by the RAG are also accurate.


## Setting Up a Python Virtual Environment in Visual Studio Code
1. Open the Command Palette (`Ctrl+Shift+P`).
2. Search for `Python: Create Environment`.
3. Select `Venv`.
4. Select a Python interpreter. Choose 3.10 or later.

> Note: The setup can take a minute. If you encounter any issues, refer to [Python environments in VS Code](https://code.visualstudio.com/docs/python/environments).

## Installing Packages
Run the required commands to install packages. If you encounter an OS permission error, add `--user` to the command line.

In [ ]:
! pip install llama-index
! pip install azure-identity
! pip install python-dotenv
! pip install trulens-eval
! pip install llama-index-vector-stores-azureaisearch
! pip install azure-search-documents --pre --upgrade

In [3]:
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from dotenv import load_dotenv
from IPython.display import Markdown, display
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.settings import Settings
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement, MetadataIndexFieldType
import logging
import os
import sys

In [4]:
# Load environment variables
load_dotenv()

# Environment Variables
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_VERSION = "2024-02-01"
AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")
AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME")
SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
SEARCH_SERVICE_API_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")
SEARCH_SERVICE_API_VERSION = "2023-11-01"
INDEX_NAME = "contoso-hr-docs"


In [5]:
# Initialize the Azure OpenAI and embedding models
llm = AzureOpenAI(
    model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
    deployment_name=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_API_VERSION,
)

embed_model = AzureOpenAIEmbedding(
    model=AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
    deployment_name=AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_API_VERSION,
)

credential = AzureKeyCredential(SEARCH_SERVICE_API_KEY)
index_client = SearchIndexClient(
    endpoint=SEARCH_SERVICE_ENDPOINT,
    credential=credential,
)

search_client = SearchClient(
    endpoint=SEARCH_SERVICE_ENDPOINT,
    index_name=INDEX_NAME,
    credential=credential,
)
Settings.llm = llm
Settings.embed_model = embed_model

In [6]:
# Initialize the vector store
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=INDEX_NAME,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="text",
    embedding_field_key="embedding",
    embedding_dimensionality=1536,
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
)

In [7]:
# Configure text splitter and extractors
from llama_index.core.extractors import TitleExtractor, QuestionsAnsweredExtractor
from llama_index.core.node_parser import TokenTextSplitter

text_splitter = TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=128)


In [16]:
# Load documents and create vector store index
import nest_asyncio
nest_asyncio.apply()

documents = SimpleDirectoryReader("data/pdf").load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, transformations=[text_splitter], storage_context=storage_context
)

In [8]:
# Load existing index from vector store
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents([], storage_context=storage_context)

In [9]:
# Query execution
query = "Does my health plan cover scuba diving?"
query_engine = index.as_query_engine(llm, similarity_top_k=3)
response = query_engine.query(query)
# Print the response
print(response)

Your health plan may not cover scuba diving, as it is important to review the plan's evidence of coverage to ensure that the service or treatment is covered under the plan. Additionally, it is advisable to discuss the service or treatment with your doctor to ensure that it is medically necessary. If scuba diving is not covered under the plan, you should discuss payment options with your doctor or healthcare provider, and consider other payment sources such as private insurance, flexible spending accounts, or state or federal programs.


In [10]:
# Compare different query engines
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode

keyword_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SPARSE, similarity_top_k=10)
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID, similarity_top_k=10)
semantic_hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID, similarity_top_k=10)

keyword_query_engine = RetrieverQueryEngine(retriever=keyword_retriever)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)
semantic_hybrid_query_engine = RetrieverQueryEngine(retriever=semantic_hybrid_retriever)

# Query and print response and source nodes for each query engine
for engine_name, engine in zip(['Keyword', 'Hybrid', 'Semantic Hybrid'], 
                               [keyword_query_engine, hybrid_query_engine, semantic_hybrid_query_engine]):
    response = engine.query(query)
    print(f"{engine_name} Response:", response)
    print(f"{engine_name} Source Nodes:")
    for node in response.source_nodes:
        print(node)
    print("\n")

Keyword Response: Your health plan covers scuba diving lessons as part of the benefits program.
Keyword Source Nodes:
Node ID: adae21cd-d890-471e-b92d-3665a4b70f15
Text: Overview   Introducing PerksPlus - the ultimate benefits program
designed to support the health and wellness of  employees. With
PerksPlus, employees have the opportunity to expense up to $1000 for
fitness -related  programs, making it easier and more affordable to
main tain a healthy lifestyle.  PerksPlus is not only  designed to
support employ...
Score:  6.646

Node ID: dd64a5c6-2b2b-4450-b4c4-161dd9449c29
Text: Not all mastectomies will qualify for WHCRA coverage. For
example, mastectomies that are  done  for cosmetic reasons or for the
treatment of a non -invasive breast cancer (i.e. Ductal  Carcinoma in
situ) are not covered under WHCRA.   In order for the coverage to be
effective, the attending physician must provide written  certification
that the ...
Score:  4.668

Node ID: 63eb9365-4d17-45ae-84f0-c169310ac9c0


## Evaluate RAG using TruLens

In [11]:
from trulens_eval import Tru
tru = Tru()
tru.reset_database()

In [15]:
from trulens_eval import Tru, Feedback, TruLlama, AzureOpenAI
import numpy as np
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from trulens_eval.app import App

provider = AzureOpenAI(
    deployment_name=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_API_VERSION,
)

context = App.select_context(query_engine)

f_groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons)
    .on(context.collect())  # collect context chunks into a list
    .on_output()
)
f_answer_relevance = Feedback(provider.relevance).on_input_output()
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons)
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

feedbacks = [f_context_relevance, f_answer_relevance, f_groundedness]

def get_prebuilt_trulens_recorder(query_engine, app_id):
    tru_recorder = TruLlama(query_engine, app_id=app_id, feedbacks=feedbacks)
    return tru_recorder

✅ In groundedness_measure_with_cot_reasons, input source will be set to __record__.app.query.rets.source_nodes[:].node.text.collect() .
✅ In groundedness_measure_with_cot_reasons, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In context_relevance_with_cot_reasons, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In context_relevance_with_cot_reasons, input context will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [13]:
keyword_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SPARSE, similarity_top_k=10)
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID, similarity_top_k=10)
semantic_hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID, similarity_top_k=10)

keyword_query_engine = RetrieverQueryEngine(retriever=keyword_retriever)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)
semantic_hybrid_query_engine = RetrieverQueryEngine(retriever=semantic_hybrid_retriever)

eval_questions = []
with open("eval/eval_contoso_hr.txt", "r") as file:
    for line in file:
        item = line.strip()
        eval_questions.append(item)

In [ ]:
# Query execution and recording for all query engines
def execute_and_record_queries(query_engine, app_id, eval_questions):
    tru_recorder = get_prebuilt_trulens_recorder(query_engine, app_id=app_id)
    with tru_recorder as recording:
        for question in eval_questions:
            query_engine.query(question)

execute_and_record_queries(keyword_query_engine, "baseline", eval_questions)
execute_and_record_queries(hybrid_query_engine, "hybrid", eval_questions)
execute_and_record_queries(semantic_hybrid_query_engine, "semantic_hybrid", eval_questions)


In [17]:
# Run dashboard to view results
tru.get_leaderboard(app_ids=["baseline", "hybrid", "semantic_hybrid"])
# tru.run_dashboard()

,groundedness_measure_with_cot_reasons,relevance,context_relevance_with_cot_reasons,latency,total_cost
app_id,,,,,
semantic_hybrid,0.718439,0.969,0.686005,7.83,0.003330
hybrid,0.641838,0.959,0.675129,7.83,0.003063
baseline,0.600933,0.953,0.671587,7.83,0.003808


In [ ]:
tru.run_dashboard()